# Exercise - Forward Pricing and Carry

#### Notation Commands

$$\newcommand{\Black}{\mathcal{B}}
\newcommand{\Blackcall}{\Black_{\mathrm{call}}}
\newcommand{\Blackput}{\Black_{\mathrm{put}}}
\newcommand{\EcondS}{\hat{S}_{\mathrm{conditional}}}
\newcommand{\Efwd}{\mathbb{E}^{T}}
\newcommand{\Ern}{\mathbb{E}^{\mathbb{Q}}}
\newcommand{\Tfwd}{T_{\mathrm{fwd}}}
\newcommand{\Tunder}{T_{\mathrm{bond}}}
\newcommand{\accint}{A}
\newcommand{\carry}{\widetilde{\cpn}}
\newcommand{\cashflow}{C}
\newcommand{\convert}{\phi}
\newcommand{\cpn}{c}
\newcommand{\ctd}{\mathrm{CTD}}
\newcommand{\disc}{Z}
\newcommand{\done}{d_{1}}
\newcommand{\dt}{\Delta t}
\newcommand{\dtwo}{d_{2}}
\newcommand{\flatvol}{\sigma_{\mathrm{flat}}}
\newcommand{\flatvolT}{\sigma_{\mathrm{flat},T}}
\newcommand{\float}{\mathrm{flt}}
\newcommand{\freq}{m}
\newcommand{\futprice}{\mathcal{F}(t,T)}
\newcommand{\futpriceDT}{\mathcal{F}(t+h,T)}
\newcommand{\futpriceT}{\mathcal{F}(T,T)}
\newcommand{\futrate}{\mathscr{f}}
\newcommand{\fwdprice}{F(t,T)}
\newcommand{\fwdpriceDT}{F(t+h,T)}
\newcommand{\fwdpriceT}{F(T,T)}
\newcommand{\fwdrate}{f}
\newcommand{\fwdvol}{\sigma_{\mathrm{fwd}}}
\newcommand{\fwdvolTi}{\sigma_{\mathrm{fwd},T_i}}
\newcommand{\grossbasis}{B}
\newcommand{\hedge}{\Delta}
\newcommand{\ivol}{\sigma_{\mathrm{imp}}}
\newcommand{\logprice}{p}
\newcommand{\logyield}{y}
\newcommand{\mat}{(n)}
\newcommand{\nargcond}{d_{1}}
\newcommand{\nargexer}{d_{2}}
\newcommand{\netbasis}{\tilde{\grossbasis}}
\newcommand{\normcdf}{\mathcal{N}}
\newcommand{\notional}{K}
\newcommand{\pfwd}{P_{\mathrm{fwd}}}
\newcommand{\pnl}{\Pi}
\newcommand{\price}{P}
\newcommand{\probexer}{\hat{\mathcal{P}}_{\mathrm{exercise}}}
\newcommand{\pvstrike}{K^*}
\newcommand{\refrate}{r^{\mathrm{ref}}}
\newcommand{\rrepo}{r^{\mathrm{repo}}}
\newcommand{\spotrate}{r}
\newcommand{\spread}{s}
\newcommand{\strike}{K}
\newcommand{\swap}{\mathrm{sw}}
\newcommand{\swaprate}{\cpn_{\swap}}
\newcommand{\tbond}{\mathrm{fix}}
\newcommand{\ttm}{\tau}
\newcommand{\value}{V}
\newcommand{\vega}{\nu}
\newcommand{\years}{\tau}
\newcommand{\yearsACT}{\tau_{\mathrm{act/360}}}
\newcommand{\yield}{Y}$$

---

# 1. Forward Pricing Fundamentals

### Data

Consider a Treasury note with the following characteristics:

| Field | Value |
|-------|-------|
| Coupon | 4.00% |
| Maturity | 2030-02-28 |
| Issue Date | 2023-02-28 |
| First Coupon | 2023-08-31 |

Market conditions as of **2023-04-18**:

| Field | Value |
|-------|-------|
| Spot Price | 102-02 (i.e., 102 + 2/32) |
| Repo Rate | 4.85% |
| Forward Date | 2023-08-01 |

In [15]:
import pandas as pd
import numpy as np
from datetime import date

In [16]:
# Bond specifications
date_issue = date(2023, 2, 28)
date_maturity = date(2030, 2, 28)
date_coupon = date(2023, 8, 31)
cpn = 0.04
face = 100

# Market conditions
date_now = date(2023, 4, 18)
date_fwd = date(2023, 8, 1)
spot_price = 102 + 2/32
repo_rate = 0.0485

### 1.1 Accrued Interest

Calculate the accrued interest at:
- Time $t$ (the current date)
- Time $T$ (the forward date)

#### Tip: Accrued Interest Formula

$$\accint_t = \frac{\cpn \cdot N}{2} \times \frac{\text{days since last coupon}}{\text{days in coupon period}}$$

where $N$ is the face value (100).

In [17]:
# Accrued interest at t (date_now) and at T (date_fwd)

def accrued_interest(cpn_rate, face_value, last_cpn_date, next_cpn_date, asof_date):
    # Accrued interest for a semi-annual coupon bond using ACT/ACT within the coupon period.
    days_since = (asof_date - last_cpn_date).days
    days_period = (next_cpn_date - last_cpn_date).days
    return (cpn_rate * face_value / 2) * (days_since / days_period), days_since, days_period

last_coupon = date_issue            # last coupon before date_now (issue date is a coupon date here)
next_coupon = date_coupon           # next coupon after date_now

accint_t, days_since_t, days_period = accrued_interest(cpn, face, last_coupon, next_coupon, date_now)
accint_T, days_since_T, _ = accrued_interest(cpn, face, last_coupon, next_coupon, date_fwd)

yearsACT = (date_fwd - date_now).days / 360  # ACT/360 for repo financing interval

print(f"Last coupon: {last_coupon} | Next coupon: {next_coupon}")
print(f"Accrued @ t={date_now}: {accint_t:.6f}  (days since last: {days_since_t}, days in period: {days_period})")
print(f"Accrued @ T={date_fwd}: {accint_T:.6f}  (days since last: {days_since_T}, days in period: {days_period})")
print(f"Forward interval (ACT/360 years): {yearsACT:.6f}")


Last coupon: 2023-02-28 | Next coupon: 2023-08-31
Accrued @ t=2023-04-18: 0.532609  (days since last: 49, days in period: 184)
Accrued @ T=2023-08-01: 1.673913  (days since last: 154, days in period: 184)
Forward interval (ACT/360 years): 0.291667


### 1.2 Forward Price

Calculate the forward price of the bond for delivery at time $T$.

#### Tip: Forward Price Formula (No Interim Coupon)

$$\fwdprice = (P_t + \accint_t)(1 + \rrepo \cdot \yearsACT) - \accint_T$$

where:
- $P_t$ is the clean spot price
- $\accint_t$ is accrued interest at $t$
- $\rrepo$ is the repo rate
- $\yearsACT$ is the forward interval in ACT/360 years
- $\accint_T$ is accrued interest at $T$

In [18]:
# Forward price (no interim coupon between date_now and date_fwd)

spot_dirty = spot_price + accint_t
fwd_price = spot_dirty * (1 + repo_rate * yearsACT) - accint_T  # forward *clean* price

print(f"Spot clean price P_t: {spot_price:.6f}")
print(f"Spot dirty price P_t^dirty: {spot_dirty:.6f}")
print(f"Forward clean price for delivery at T={date_fwd}: {fwd_price:.6f}")

Spot clean price P_t: 102.062500
Spot dirty price P_t^dirty: 102.595109
Forward clean price for delivery at T=2023-08-01: 102.372489


### 1.3 The Forward Drop

Calculate the **forward drop**, which is the difference between the spot and forward prices.

Is the forward price higher or lower than the spot? Explain why, in terms of the carry relationship.

In [19]:
# Forward drop: spot minus forward

forward_drop = spot_price - fwd_price

direction = "higher" if fwd_price > spot_price else "lower"
print(f"Forward drop (P_t - F): {forward_drop:.6f}")
print(f"The forward price is {direction} than the spot clean price.")

print("\nIntuition:")
print("- Carry over (t,T) is roughly coupon income minus financing cost.")
print("- Here repo_rate > coupon rate, so carry is negative, which pushes the forward price ABOVE spot.")

Forward drop (P_t - F): -0.309989
The forward price is higher than the spot clean price.

Intuition:
- Carry over (t,T) is roughly coupon income minus financing cost.
- Here repo_rate > coupon rate, so carry is negative, which pushes the forward price ABOVE spot.


---

# 2. Carry Analysis

### 2.1 Calculate Carry

Compute the **carry** of holding this bond from $t$ to $T$.

#### Tip: Carry Formula

$$\text{Carry} = \left(\cpn \cdot N - \rrepo \cdot P_t^{\text{dirty}}\right) \cdot \yearsACT$$

where $P_t^{\text{dirty}} = P_t + \accint_t$.

Alternatively, expressed as a rate:
$$\carry = \cpn - \rrepo$$

In [20]:
# Carry of holding the bond from t to T (dollar carry over the interval)

carry = (cpn * face - repo_rate * spot_dirty) * yearsACT
carry_rate = cpn - repo_rate

print(f"Carry (dollars over (t,T)): {carry:.6f}")
print(f"Carry rate (approx): {carry_rate:.6%}  (coupon - repo)")

Carry (dollars over (t,T)): -0.284627
Carry rate (approx): -0.850000%  (coupon - repo)


### 2.2 Verify the Relationship

Verify that the forward drop approximately equals the carry:

$$P_t - \fwdprice \approx \text{Carry}$$

Why might this relationship not hold exactly?

In [21]:
# Verify: forward drop ≈ carry

print(f"Forward drop (P_t - F): {forward_drop:.6f}")
print(f"Carry (over (t,T)):     {carry:.6f}")
print(f"Difference:             {(forward_drop - carry):.6f}")

print("\nWhy not exact?")
print("- Day count conventions differ (ACT/ACT for accrued vs ACT/360 for repo).")
print("- Accrued interest is computed within a coupon period; financing uses a money-market convention.")
print("- Rounding/market conventions (32nds, settlement conventions) can introduce small differences.")

Forward drop (P_t - F): -0.309989
Carry (over (t,T)):     -0.284627
Difference:             -0.025362

Why not exact?
- Day count conventions differ (ACT/ACT for accrued vs ACT/360 for repo).
- Accrued interest is computed within a coupon period; financing uses a money-market convention.
- Rounding/market conventions (32nds, settlement conventions) can introduce small differences.


---

# 3. Forward Pricing with Interim Coupon

Now consider a forward contract that spans across a coupon date.

**New scenario:**
- Current date: 2023-04-18
- Forward date: 2023-10-15 (after the August coupon)
- Coupon payment date: 2023-08-31

The bond still has:
- Coupon: 4%
- Spot price: 102-02
- Repo rate: 4.85%

In [22]:
# New forward date (after coupon)
date_fwd_new = date(2023, 10, 15)
date_next_coupon = date(2023, 8, 31)

### 3.1 Forward Price with Interim Coupon

Calculate the forward price when there is an interim coupon payment.

#### Tip: Forward Price with Interim Coupon

$$\fwdprice = \left[(P_t + \accint_t)(1 + \rrepo \cdot \tau_0) - \frac{\cpn \cdot N}{2}\right](1 + \rrepo \cdot \tau_1) - \accint_T$$

where:
- $\tau_0$ is the pre-coupon interval (ACT/360)
- $\tau_1$ is the post-coupon interval (ACT/360)

In [23]:
# Forward price when there IS an interim coupon payment

# Intervals in ACT/360 years
tau0 = (date_next_coupon - date_now).days / 360      # t -> coupon
tau1 = (date_fwd_new - date_next_coupon).days / 360  # coupon -> T_new

coupon_payment = cpn * face / 2

# Accrued at the new forward date: last coupon is date_next_coupon; next coupon is ~6 months later
date_after_coupon = date(2024, 2, 28)  # next coupon after 2023-08-31 (end-Feb schedule)
accint_Tnew, _, days_period2 = accrued_interest(cpn, face, date_next_coupon, date_after_coupon, date_fwd_new)

fwd_price_with_coupon = ((spot_dirty * (1 + repo_rate * tau0) - coupon_payment) * (1 + repo_rate * tau1) - accint_Tnew)

print(f"Coupon date crossed: {date_next_coupon} (coupon = {coupon_payment:.2f})")
print(f"Accrued @ T_new={date_fwd_new}: {accint_Tnew:.6f}  (days in period: {days_period2})")
print(f"Forward clean price with interim coupon: {fwd_price_with_coupon:.6f}")

Coupon date crossed: 2023-08-31 (coupon = 2.00)
Accrued @ T_new=2023-10-15: 0.497238  (days in period: 181)
Forward clean price with interim coupon: 102.584990


### 3.2 Compare Forward Prices

How does the forward price with an interim coupon compare to a hypothetical forward price calculated without accounting for the coupon? 

Explain the economic intuition.

In [24]:
# Compare to a hypothetical "naive" forward price that ignores the coupon

tau_total = (date_fwd_new - date_now).days / 360
fwd_price_naive = spot_dirty * (1 + repo_rate * tau_total) - accint_Tnew

print(f"Naive forward (ignoring coupon): {fwd_price_naive:.6f}")
print(f"Correct forward (with coupon):  {fwd_price_with_coupon:.6f}")
print(f"Naive - Correct:                {(fwd_price_naive - fwd_price_with_coupon):.6f}")

print("\nEconomic intuition:")
print("- If you ignore the coupon, you act like the bond 'keeps' that cash until delivery.")
print("- In reality, the holder receives the coupon on the coupon date, so the forward buyer does NOT get it.")
print("- Therefore the correct forward price is LOWER by roughly the coupon amount (grown at repo) over the remaining time.")

Naive forward (ignoring coupon): 104.585803
Correct forward (with coupon):  102.584990
Naive - Correct:                2.000813

Economic intuition:
- If you ignore the coupon, you act like the bond 'keeps' that cash until delivery.
- In reality, the holder receives the coupon on the coupon date, so the forward buyer does NOT get it.
- Therefore the correct forward price is LOWER by roughly the coupon amount (grown at repo) over the remaining time.


---

# 4. Forward vs Futures

### 4.1 Daily Settlement Impact

Suppose the Treasury futures contract on this bond is trading at 101.50.

Given your calculated forward price, is the futures price higher or lower than the forward price?

Explain why Treasury futures prices are typically **lower** than forward prices due to the daily settlement (mark-to-market) mechanism.

In [25]:
# Forward vs futures comparison

futures_price = 101.50

print(f"Forward clean price (from 1.2, delivery {date_fwd}): {fwd_price:.6f}")
print(f"Futures price given:                               {futures_price:.6f}")

rel = "higher" if futures_price > fwd_price else "lower"
print(f"Futures price is {rel} than the forward price.")

print("\nWhy futures are typically lower than forwards (daily settlement):")
print("- Futures are marked-to-market daily; gains/losses are realized and effectively reinvested/financed day by day.")
print("- This changes the effective financing vs a single forward settlement at T, especially when rates are uncertain.")
print("- With positive funding costs, daily settlement tends to reduce the price relative to a forward (in typical Treasury settings).")

Forward clean price (from 1.2, delivery 2023-08-01): 102.372489
Futures price given:                               101.500000
Futures price is lower than the forward price.

Why futures are typically lower than forwards (daily settlement):
- Futures are marked-to-market daily; gains/losses are realized and effectively reinvested/financed day by day.
- This changes the effective financing vs a single forward settlement at T, especially when rates are uncertain.
- With positive funding costs, daily settlement tends to reduce the price relative to a forward (in typical Treasury settings).


### 4.2 Tailing the Hedge

If you want to hedge a $10 million notional forward position using futures, how many futures contracts should you use?

#### Tip: Tailing the Hedge

$$N^{\text{fut}} = Z(t,T) \cdot N^{\text{fwd}}$$

where $Z(t,T) = \frac{1}{1 + \rrepo \cdot \yearsACT}$

Assume each futures contract has a notional of $100,000.

In [26]:
# Tailing the hedge

N_fwd_notional = 10_000_000
contract_notional = 100_000

N_fwd_contracts = N_fwd_notional / contract_notional  # "untailored" hedge
Z = 1 / (1 + repo_rate * yearsACT)                    # discount factor over (t,T) using simple repo

N_fut = Z * N_fwd_contracts

print(f"Repo discount factor Z(t,T): {Z:.6f}")
print(f"Forward notional:            ${N_fwd_notional:,.0f}")
print(f"Futures contract notional:   ${contract_notional:,.0f}")
print(f"Untailed hedge:              {N_fwd_contracts:.2f} contracts")
print(f"Tailed hedge:                {N_fut:.2f} contracts")

Repo discount factor Z(t,T): 0.986051
Forward notional:            $10,000,000
Futures contract notional:   $100,000
Untailed hedge:              100.00 contracts
Tailed hedge:                98.61 contracts


***